In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

import spacy

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from collections import defaultdict, Counter
from tqdm import tqdm

import re

import torch
from torch import nn
from torch.utils.data import DataLoader

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_split_aspects.txt
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_split_reviews.txt
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/dev_aspects.txt
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/dev_reviews.txt

--2021-12-27 11:39:21--  https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_split_aspects.txt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_split_aspects.txt [following]
--2021-12-27 11:39:21--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_split_aspects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170883 (167K) [text/plain]
Saving to: ‘train_split_aspects.txt’

train_split_aspects 100%[===================>] 166.88K  --.-KB/s    in 0.02s   

2021-12-27 11:39:21 (7.64 MB/s) - 

In [3]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 24.4 MB/s 
     |████████████████████████████████| 61 kB 77 kB/s 
     |████████████████████████████████| 895 kB 26.1 MB/s 
     |████████████████████████████████| 3.3 MB 29.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
train_asp = pd.read_csv('train_split_aspects.txt', 
                        delimiter='\t', 
                        names=['text_id', 'category', 'mention', 
                               'start', 'end', 'sentiment'])

train_texts = pd.read_csv('train_split_reviews.txt', 
                          delimiter='\t', names=['text_id','text'])

In [5]:
data = train_asp.merge(train_texts, on='text_id')

In [6]:
data

,text_id,category,mention,start,end,sentiment,text
0,30808,Whole,ресторане,16,25,neutral,Отмечали в этом ресторане день рождение на пер...
1,30808,Interior,первом этаже,43,55,neutral,Отмечали в этом ресторане день рождение на пер...
2,30808,Whole,руководству ресторана,124,145,positive,Отмечали в этом ресторане день рождение на пер...
3,30808,Service,обслуживающему персоналу,147,171,positive,Отмечали в этом ресторане день рождение на пер...
4,30808,Service,сотрудникам,189,200,positive,Отмечали в этом ресторане день рождение на пер...
...,...,...,...,...,...,...,...
3568,16630,Service,обслуживание,85,97,positive,Уютная и тёплая домашняя обстановка! Милый и о...
3569,16630,Food,Еда,99,102,positive,Уютная и тёплая домашняя обстановка! Милый и о...
3570,16630,Service,персоналу,244,253,positive,Уютная и тёплая домашняя обстановка! Милый и о...
3571,16630,Whole,ресторан,294,302,positive,Уютная и тёплая домашняя обстановка! Милый и о...


In [7]:
data['sentiment'].value_counts()

positive    2309
neutral      694
negative     500
both          70
Name: sentiment, dtype: int64

In [11]:
temp = data.drop(data.query('sentiment ==  "positive"').sample(frac=.6).index)

In [12]:
temp['sentiment'].value_counts()

positive    924
neutral     694
negative    500
both         70
Name: sentiment, dtype: int64

In [13]:
sentiment_vocab = {'positive': 1, 'negative': 0, 'both': 2, 'neutral': 3}

In [14]:
class Dataset():

    def __init__(self, df):
        self.df = df

    def __getitem__(self, index):
        text = self.df['text'].values[index]
        sentiment = self.df['sentiment'].values[index]
        masked = (text[:self.df['start'].values[index]] + '[MASK]' + 
                  text[self.df['end'].values[index]:])
        
        return masked, sentiment_vocab[sentiment]

    def __len__(self):

        return len(self.df)

In [15]:
train_data, val_data = train_test_split(temp, test_size=0.25, stratify=temp.sentiment.values)

train_dataset = Dataset(train_data)
val_dataset = Dataset(val_data)

In [16]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [17]:
class Classifier(nn.Module):

    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.dropout = nn.Dropout(p=0.3)
        self.linear_1 = nn.Linear(312, 4)
        self.softmax = nn.Softmax()

    def forward(self, x, attention_mask=None):
        output  = self.bert(x, attention_mask=attention_mask)
        x = output['last_hidden_state'][:, 0]
        x = self.dropout(x)
        x = self.linear_1(x)
        x = self.softmax(x)
        
        return x

In [18]:
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup

In [19]:
MODEL_NAME = "cointegrated/rubert-tiny"
EPOCHS = 4
BATCH_SIZE = 8

In [20]:
train_loss = []
val_loss = []

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
criterion = nn.CrossEntropyLoss()

In [21]:
model = AutoModel.from_pretrained(
    MODEL_NAME
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, 
    do_lower_case=True
)

optimizer = AdamW(model.parameters(),
                  weight_decay = 1e-4,
                  lr = 4e-5,
                  eps = 1e-8
                )

total_steps = len(train_dataloader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.5M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [22]:
model_classifier = Classifier(model)

In [23]:
from sklearn.metrics import accuracy_score, f1_score

In [24]:
import random
import numpy as np

seed_val = 2020

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

for epoch in tqdm(range(EPOCHS)):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, EPOCHS))
    print('Training...')

    current_train_loss = 0 
    model_classifier.train()

    for text, y in tqdm(train_dataloader, leave=False):
        tokens = tokenizer.batch_encode_plus(list(text),
            pad_to_max_length=True,
            return_tensors='pt')

        logits = model_classifier(tokens['input_ids'], 
                                  attention_mask=tokens['attention_mask'])

        loss = criterion(logits, y)
        current_train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model_classifier.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    
    train_loss.append(current_train_loss / len(train_dataloader))

    model_classifier.eval()
    current_val_loss = 0
    predictions = []
    actuals = [] 

    for text, y in val_dataloader:
        tokens = tokenizer.batch_encode_plus(
            list(text),
            pad_to_max_length=True,
            return_tensors='pt')

        logits = model_classifier(tokens['input_ids'], 
                                  attention_mask=tokens['attention_mask'])
        
        loss = criterion(logits, y)
        current_val_loss += loss.item()

        predictions += list(logits.argmax(axis=1).data.numpy())
        actuals += list(y.data.numpy())
    
    print('Accuracy:', accuracy_score(actuals, predictions))    
    val_loss.append(current_val_loss / len(val_dataloader))

  0%|          | 0/4 [00:00<?, ?it/s]


======== Epoch 1 / 4 ========
Training...



  0%|          | 0/206 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app

 25%|██▌       | 1/4 [09:30<28:31, 570.36s/it]

Accuracy: 0.48811700182815354

======== Epoch 2 / 4 ========
Training...



  0%|          | 0/206 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app

 50%|█████     | 2/4 [18:44<18:42, 561.06s/it]

Accuracy: 0.5118829981718465

======== Epoch 3 / 4 ========
Training...



  0%|          | 0/206 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app

 75%|███████▌  | 3/4 [27:58<09:17, 557.84s/it]

Accuracy: 0.5191956124314442

======== Epoch 4 / 4 ========
Training...



  0%|          | 0/206 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app

100%|██████████| 4/4 [37:12<00:00, 558.19s/it]

Accuracy: 0.5319926873857403


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.save(model_classifier.state_dict(), '/content/drive/My Drive/sentiment_classifier')

In [27]:
model_classifier

Classifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29564, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
 

In [28]:
test_asp = pd.read_csv('dev_aspects.txt', 
                        delimiter='\t', 
                        names=['text_id', 'category', 'mention', 
                               'start', 'end', 'sentiment'])

test_texts = pd.read_csv('dev_reviews.txt', 
                          delimiter='\t', names=['text_id','text'])

In [29]:
test_data = test_asp.merge(test_texts, on='text_id')

In [30]:
test_data

,text_id,category,mention,start,end,sentiment,text
0,3976,Whole,ресторане,71,80,neutral,"День 8-го марта прошёл, можно и итоги подвести..."
1,3976,Whole,ресторанах,198,208,neutral,"День 8-го марта прошёл, можно и итоги подвести..."
2,3976,Whole,ресторане,256,265,neutral,"День 8-го марта прошёл, можно и итоги подвести..."
3,3976,Service,Столик бронировали,267,285,neutral,"День 8-го марта прошёл, можно и итоги подвести..."
4,3976,Service,администратор,322,335,positive,"День 8-го марта прошёл, можно и итоги подвести..."
...,...,...,...,...,...,...,...
1185,33043,Service,заказ,792,797,positive,Мне так там нравитсяяяя!!!!!!!!! Интерьер модн...
1186,33043,Service,принесли,798,806,positive,Мне так там нравитсяяяя!!!!!!!!! Интерьер модн...
1187,33043,Food,приготовили,880,891,positive,Мне так там нравитсяяяя!!!!!!!!! Интерьер модн...
1188,33043,Service,оставил,1017,1024,negative,Мне так там нравитсяяяя!!!!!!!!! Интерьер модн...


In [31]:
test_dataset = Dataset(test_data)

In [32]:
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [33]:
predictions = []
true_sentiment = []

print('Predicting sentiment for {:,} sentiment mention...'.format(len(test_data)))

model_classifier.eval()

for text, y in tqdm(test_dataloader):
    tokens = tokenizer.batch_encode_plus(list(text),
        pad_to_max_length=True,
        return_tensors='pt')

    logits = model_classifier(tokens['input_ids'],
        attention_mask=tokens['attention_mask']).argmax(dim=1)
    
    predictions += list(logits.data.numpy())
    true_sentiment += list(y.data.numpy())

Predicting sentiment for 1,190 sentiment mention...


  0%|          | 0/149 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app
100%|██████████| 149/149 [01:23<00:00,  1.77it/s]


In [34]:
print('Accuracy:', accuracy_score(true_sentiment, predictions))

Accuracy: 0.5815126050420169


In [35]:
pred_data = pd.read_csv('dev_aspects.txt', 
                        delimiter='\t', 
                        names=['text_id', 'category', 'mention', 
                               'start', 'end', 'sentiment'])

In [36]:
pred_data['pred_sentiment'] = predictions

In [37]:
pred_data['pred_sentiment'] = pred_data['pred_sentiment'].replace([0, 1, 2, 3], 
                                                   ['negative', 
                                                    'positive', 
                                                    'both',
                                                    'neutral'])

In [38]:
pred_data.pred_sentiment.unique()

array(['positive', 'negative', 'neutral'], dtype=object)

In [39]:
pred_data = pred_data.drop('sentiment', axis=1)

In [40]:
pred_data = pred_data.rename(columns={'pred_sentiment': 'sentiment'})

In [41]:
pred_data

,text_id,category,mention,start,end,sentiment
0,3976,Whole,ресторане,71,80,positive
1,3976,Whole,ресторанах,198,208,positive
2,3976,Whole,ресторане,256,265,positive
3,3976,Service,Столик бронировали,267,285,positive
4,3976,Service,администратор,322,335,positive
...,...,...,...,...,...,...
1185,33043,Service,заказ,792,797,positive
1186,33043,Service,принесли,798,806,positive
1187,33043,Food,приготовили,880,891,positive
1188,33043,Service,оставил,1017,1024,positive


In [42]:
pred_data.to_csv('/content/drive/My Drive/dev_pred_aspects.txt')